# Benchmarking NumPy and CuPy

* CuPy is an open source library for GPU-accelerated computing with Python. It shares the same API set as NumPy and SciPy, allowing it to be a drop-in replacement to run NumPy/SciPy code on GPU.
* "Cu" in CuPy stands for Nvidia's CUDA framework which allows program to perform general-purpose calculations on GPU.

In [1]:
import numpy as np
import time

def generate_matrices(m: int, k: int, n: int):
    A = np.random.uniform(-1, 1, size=(m, k)).astype(np.float32)
    B = np.random.uniform(-1, 1, size=(k, n)).astype(np.float32)
    return (A, B)

# Dimensions of matrices
m = 37000
k = 23000
n = 18000

# some magic numbers, just to make calculation heavier
sqrt2 = 1.414
pi = 3.1415

In [2]:
A, B = generate_matrices(m, k, n)
print(A.shape)
print(B.shape)

(37000, 23000)
(23000, 18000)


## NumPy version

In [3]:
np_A = A.copy()
np_B = B.copy()

t0 = time.time()
np_A = np.log(np_A + sqrt2)
np_B *= pi
np_C = np.dot(np_A, np_B)
np_C = np.mean(np_C, axis=0)
np_C_norm = np_C / np.linalg.norm(np_C)

t1 = time.time()


In [4]:
print(len(np_C_norm))
print(np_C_norm)
print(f'{(t1 - t0):,.03f}sec')

18000
[ 0.00454275  0.00314037 -0.00701416 ...  0.00196507  0.00404032
 -0.001335  ]
74.277sec


## CuPy version

In [5]:
import cupy as cp

print('Checking GPU info...')
if cp.cuda.is_available():
    device_id = cp.cuda.runtime.getDevice()
    device_properties = cp.cuda.runtime.getDeviceProperties(device_id)
    print(f"Model:  {device_properties['name'].decode()}")
    print(f"Memory: {device_properties['totalGlobalMem']/1e9:.1f}GB")
else:
    raise RuntimeError('GPU is not available')

Checking GPU info...
Model:  NVIDIA GeForce RTX 3060
Memory: 12.6GB


In [6]:
t0 = time.time()
# Move data from main memory to GPU memory
cp_A = cp.asarray(A)
cp_B = cp.asarray(B)

t1 = time.time()
# Calculation
cp_A = cp.log(cp_A + sqrt2)
cp_B = cp_B / pi
cp_C = cp.dot(cp_A, cp_B) 
cp_C = np.mean(cp_C, axis=0)
cp_C_norm = cp_C / cp.linalg.norm(cp_C)

t2 = time.time()
# Move data from GPU memory back to CPU memory
np_C_norm2 = cp.asnumpy(cp_C_norm)

t3 = time.time()

In [7]:
print(len(np_C_norm2))
print(np_C_norm2)
print(f'Move data from main memory to GPU memory: {(t1 - t0):,.3f}sec')
print(f'Calculation:                              {(t2 - t1):,.3f}sec')
print(f'Move data from GPU memory to main memory: {(t3 - t2):,.3f}sec')
print('=====')
print(f'Total:                                    {(t3 - t0):,.3f}sec')


18000
[ 0.00454276  0.00314038 -0.00701416 ...  0.00196508  0.00404033
 -0.001335  ]
Move data from main memory to GPU memory: 2.400sec
Calculation:                              3.472sec
Move data from GPU memory to main memory: 0.000sec
=====
Total:                                    5.872sec


## Results consistency verification

In [8]:
tolerance = 1e-7
np.allclose(np_C_norm, np_C_norm2, atol=tolerance)

True

## More results

| m\*n\*k       | cuBLAS      | OpenBLAS    | CuPy       | NumPy      |
| ------------- | ----------- | ----------- | ---------- | ---------- |
| 3k\*1k\*2k    | 104.2ms     | 46.8ms      | 172.8ms    | 44.7ms     |
| 8k\*2k\*3k    | 132.3ms     | 315.6ms     | 267.5ms    | 300.2ms    |
| 17k\*4k\*6k   | 0.361s      | 2.292s      | 0.596s     | 2.165s     |
| 25k\*6k\*8k   | 0.631s      | 6.486s      | 1.111s     | 6.108s     |
| 30k\*8k\*11k  | 1.145s      | 13.847s     | 1.704s     | 13.336s    |
| 50k\*9k\*13k  | 2.097s      | 30.249s     | 3.279s     | 28.735s    |
| 77k\*12k\*23k | 6.336s      | 106.817s    | [OOM]      | [OOM]      |

Notes:
* OpenBLAS is the typical linear algebra library internally used by NumPy.
* cuBLAS is CuPy's equivalent implemented in C.